In [ ]:
import glob, os, pickle

import gc

import AE as ae

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Load MAEs

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where precomputed MAEs generated by AE Gen MAEs.ipynb are saved.

mae_files = glob.glob(os.path.join(directory, '[0-9]*_maes.pkl')) # Use patterns to detect all the saved files.

SNRs = [float(x.split('/')[-1].split('_')[0]) for x in mae_files]

columns = ['SNR', 'mae_file']
# Create DataFrame of MAE file locations
mae_df = pd.DataFrame(dict(zip(columns, [SNRs, mae_files])))
mae_df

In [ ]:
data_directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

data_files = glob.glob(os.path.join(data_directory, '*_SNR_data.pkl')) # Use patterns to detect all the saved files.
SNRs = [float(x.split('/')[-1].split('_')[0]) for x in data_files]

columns = ['SNR', 'data_file']
# Create DataFrame of Sim file locations
data_df = pd.DataFrame(dict(zip(columns, [SNRs, data_files])))
data_df

In [ ]:
# Merge DataFrames
df = mae_df.set_index('SNR').join(data_df.set_index('SNR'))
df

In [ ]:
stat_dir = 'INSERT DIRECTORY' # Directory where background analysis results generated by AE Background.ipynb are saved.

# Load threshold values
with open(os.path.join(stat_dir, 'INSERT FILENAME'), 'rb') as f:
    thresholds_dict = pickle.load(f)
    f.close()
thresholds_dict

In [ ]:
#Initialise constants to match those used in AE Gen MAEs.ipynb
window_size = 5 
step_size = 5

## Event Search

In [ ]:
columns = ['SNR', 'Thresh', 'event_locations']
evt_df = pd.DataFrame(columns=columns)

SNRs = df.index.values
thresholds = list(thresholds_dict.keys())
for snr in SNRs:
    data_file = df.loc[snr]['data_file']
    mae_file = df.loc[snr]['mae_file']
    
    print('loading maes')
    # Load precomputed MAEs
    with open(mae_file, 'rb') as f:
        traces_maes = pickle.load(f)
        f.close()
    print('loaded')
    
    # For each threshold
    for i, thresh in enumerate(thresholds):
        print("SNR: " + str(snr))
        print("Threshold: " + str(thresh))
        
        thresh_val = thresholds_dict[thresh]
        
        # Locate Events
        evt_locs = ae.event_search_from_maes(traces_maes, thresh_val, window_size, step_size, prestepped=True)
        
        
        print("Event Search Finished")
        
        evt_df2 = pd.DataFrame([[snr, thresh, evt_locs]], columns=columns)
        evt_df = pd.concat((evt_df, evt_df2))

In [ ]:
evt_df = evt_df.set_index(['SNR', 'Thresh'])
evt_df = evt_df.sort_index()
evt_df

## Free Memory

In [ ]:
del truths
del traces_maes
del evt_locs
del evt_df2

In [ ]:
gc.collect()

# Save Found Events

In [ ]:
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(evt_df, f)